In [3]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np
import nltk
wordnet_lemmatizer = WordNetLemmatizer()
nltk.download('stopwords')
nltk.download('wordnet')
import os
from bs4 import BeautifulSoup
import re
from nltk.tokenize import WordPunctTokenizer
tkn = WordPunctTokenizer()


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Amara\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Amara\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [21]:
from textblob import TextBlob

In [1]:
pattern0 = r'@[A-Za-z0-9]+'
pattern1 = r'https?://[A-Za-z0-9./]+'
comb_pattern = r'|'.join((pattern0, pattern1))

def text_cleaners(text):
    sp = BeautifulSoup(text, 'lxml')
    prsd = sp.get_text() #To parse the html content
    striped = re.sub(comb_pattern, '', prsd) #To check and clean anything fall under combined pattern (comb_pattern)
    try:
        clean = striped.encode().decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        clean = stripped
    letters_only = re.sub("[^a-zA-Z]", " ", clean)
    replace_rt = letters_only.replace('RT','')
    lower_case = replace_rt.lower()
    # The 9th line of this function created unnecessay white spaces
    # To remove these white spaces we tokenize and joined tokens 
    word_list = tkn.tokenize(lower_case)
    return (" ".join(word_list)).strip()

In [4]:
#Check Current working directory
cwd = os.getcwd()
cwd

'C:\\Users\\Amara\\0_MRP'

In [5]:
#change the directory
os.chdir("C:\\#Study\\2. Ryerson_all\\THE MRP\\Dataset\\test_pad")

In [6]:
#Read canadian tire tweets csv file
df_ct = pd.read_csv("can_tire_all_final.csv", encoding="latin-1")

In [33]:
#check the loaded file
df_ct.head(2)

,text,favoriteCount,replyToSN,created,replyToSID,statusSource,screenName,retweetCount,isRetweet,retweeted,cleaned_text,blobw,sentiment,Polarity,Subjectivity,Sentiment_class
0,RT @CanadianTire: Wherever you are in the gala...,0,NaN,5/8/2018 3:48,NaN,"<a href=""https://mobile.twitter.com"" rel=""nofo...",Chris31Austin,34,True,False,wherever you are in the galaxy maythe thbewithyou,"(w, h, e, r, e, v, e, r, , y, o, u, , a, r, ...","(0.0, 0.0)",0.0,0.0,Neutral
1,RT @cmcdavid97: Were all standing behind you ...,0,NaN,5/8/2018 3:38,NaN,"<a href=""http://twitter.com/download/iphone"" r...",LucasBuzzno,182,True,False,we re all standing behind you weallplayforcanada,"(w, e, , r, e, , a, l, l, , s, t, a, n, d, ...","(-0.4, 0.7)",-0.4,0.7,Negative


In [15]:
df_ct.isDuplicated.value_counts()

False    6627
Name: isDuplicated, dtype: int64

In [17]:
#Remove unnecessary columns
df_ct.drop(['Unnamed: 0','longitude','favorited','truncated','latitude','id','isDuplicated','replyToUID','Unnamed: 18'],axis=1,inplace=True) 

In [19]:
#Apply text cleaning function to data frame
df_ct['cleaned_text'] = df_ct['text'].apply(lambda x : text_cleaners(x))

C:\Users\Amara\Anaconda2\envs\py36\lib\site-packages\bs4\__init__.py:282: UserWarning: "https://t.co/tMYuro525I" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


In [22]:
df_ct['blobw'] = df_ct['cleaned_text'].apply(TextBlob)

In [23]:
#Create a function to calculate sentiment and subjectivity
def sentiment_calc(text):
    try:
        return TextBlob(text).sentiment
    except:
        return None


In [25]:
df_ct['sentiment'] = df_ct['cleaned_text'].apply(sentiment_calc)

In [26]:
#Create a function to calculate polarity
def polarity_calc(text):
    try:
        return TextBlob(text).polarity
    except:
        return None

In [27]:
def subjectivity_calc(text):
    try:
        return TextBlob(text).subjectivity
    except:
        return None

In [30]:
df_ct['Polarity'] = df_ct['cleaned_text'].apply(polarity_calc)
df_ct['Subjectivity'] = df_ct['cleaned_text'].apply(subjectivity_calc)

In [31]:
#Create a funtion to assign sentiment measures as per polarity
def conditions(s):
    if (s['Polarity'] > 0):
        return "Positive"
    elif (s['Polarity']<0):
        return "Negative"
    else:
        return "Neutral"

In [32]:
df_ct['Sentiment_class'] = df_ct.apply(conditions, axis=1)

In [34]:
#Export Canadian Tire file to local machine
df_ct.to_csv("can_tire_senti_score.csv", encoding='utf-8', index=False)

#We have following Re-usable functions
#text_cleaners, sentiment_calc, polarity_calc, subjectivity_calc, conditions

In [43]:
# Get sentiment for walmart tweet

#Read walmart tweets csv file
df_wm = pd.read_csv("walmart_all.csv", encoding="latin-1")

In [44]:
df_wm.head(2)

,Unnamed: 0,text,favorited,favoriteCount,replyToSN,created,truncated,replyToSID,id,replyToUID,statusSource,screenName,retweetCount,isRetweet,retweeted,longitude,latitude,isDuplicated
0,1,RT @WalmartCanada: @HOSIXTAPE Hello. The BTS a...,False,0,NaN,5/8/2018 2:17,False,NaN,9.940000e+17,NaN,"<a href=""http://twitter.com/download/iphone"" r...",gr7jk9,234,True,False,NaN,NaN,False
1,2,RT @WalmartCanada: @HOSIXTAPE Hello. The BTS a...,False,0,NaN,5/8/2018 1:57,False,NaN,9.940000e+17,NaN,"<a href=""http://twitter.com/download/iphone"" r...",flyingredpig218,234,True,False,NaN,NaN,False


In [45]:
#Remove unnecessary columns
df_wm.drop(['Unnamed: 0','longitude','favorited','truncated','latitude','id','isDuplicated','replyToUID'],axis=1,inplace=True) 


In [46]:
df_wm['cleaned_text'] = df_wm['text'].apply(lambda x : text_cleaners(x))

In [47]:
df_wm['blobw'] = df_wm['cleaned_text'].apply(TextBlob)

In [48]:
df_wm['sentiment'] = df_wm['cleaned_text'].apply(sentiment_calc)

In [49]:
df_wm['Polarity'] = df_wm['cleaned_text'].apply(polarity_calc)
df_wm['Subjectivity'] = df_wm['cleaned_text'].apply(subjectivity_calc)

In [50]:
df_wm['Sentiment_class'] = df_wm.apply(conditions, axis=1)

In [51]:
#Export Canadian Tire file to local machine
df_wm.to_csv("walmart_senti_score.csv", encoding='utf-8', index=False)

# Get sentiment for walmart tweet



In [52]:
# Get sentiment for Loblaws tweet

#Read loblaws tweets csv file
df_ll = pd.read_csv("loblaws_all.csv", encoding="latin-1")

In [53]:
#check the loaded file
df_ll.head(2)

df_ll.isDuplicated.value_counts()
#Remove unnecessary columns
df_ll.drop(['Unnamed: 0','longitude','favorited','truncated','latitude','id','isDuplicated','replyToUID'],axis=1,inplace=True) 


In [56]:
df_ll.head(2)

,text,favoriteCount,replyToSN,created,replyToSID,statusSource,screenName,retweetCount,isRetweet,retweeted,cleaned_text,blobw,sentiment,Polarity,Subjectivity,Sentiment_class
0,@matthewlmcauley @LoblawsON The symbols they u...,0,matthewlmcauley,5/8/2018 3:50,9.936900e+17,"<a href=""http://twitter.com/download/iphone"" r...",KassandraSantos,0,False,False,the symbols they use aren t even clear lol i d...,"(t, h, e, , s, y, m, b, o, l, s, , t, h, e, ...","(0.4666666666666666, 0.5277777777777778)",0.466667,0.527778,Positive
1,A female engineer goes to her kid's career day...,0,NaN,5/8/2018 3:36,NaN,"<a href=""http://twitter.com/download/android"" ...",HutzMs,0,False,False,a female engineer goes to her kid s career day...,"(a, , f, e, m, a, l, e, , e, n, g, i, n, e, ...","(0.1875, 0.4583333333333333)",0.187500,0.458333,Positive


In [55]:

df_ll['cleaned_text'] = df_ll['text'].apply(lambda x : text_cleaners(x))

df_ll['blobw'] = df_ll['cleaned_text'].apply(TextBlob)

df_ll['sentiment'] = df_ll['cleaned_text'].apply(sentiment_calc)


df_ll['Polarity'] = df_ll['cleaned_text'].apply(polarity_calc)
df_ll['Subjectivity'] = df_ll['cleaned_text'].apply(subjectivity_calc)


df_ll['Sentiment_class'] = df_ll.apply(conditions, axis=1)

#Export Canadian Tire file to local machine
df_ll.to_csv("loblaws_senti_score.csv", encoding='utf-8', index=False)



C:\Users\Amara\Anaconda2\envs\py36\lib\site-packages\bs4\__init__.py:282: UserWarning: "https://t.co/QJjeg2triC" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Users\Amara\Anaconda2\envs\py36\lib\site-packages\bs4\__init__.py:282: UserWarning: "https://t.co/MR3fTbwEIA" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Users\Amara\Anaconda2\envs\py36\lib\site-packages\bs4\__init__.py:282: UserWarning: "https://t.co/vsPGmmtKTP" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beauti

In [57]:
# Get sentiment for Costco tweet

#Read costco tweets csv file
df_cs = pd.read_csv("costco_all.csv", encoding="latin-1")

In [64]:
#check the loaded file
df_cs.head(2)

,text,favoriteCount,replyToSN,created,replyToSID,statusSource,screenName,retweetCount,isRetweet,retweeted,cleaned_text,blobw,sentiment,Polarity,Subjectivity
0,@gardein but I live in Canada <ed><U+00A0><U+0...,0,gardein,2018-05-13 23:12:14,9.957750e+17,"<a href=""http://mvilla.it/fenix"" rel=""nofollow...",greazykitty,0,False,False,but i live in canada maybe you can make a dist...,"(b, u, t, , i, , l, i, v, e, , i, n, , c, ...","(0.13636363636363635, 0.5)",0.136364,0.5
1,RT @CHEOhospital: #DYK that were celebrating ...,0,NaN,2018-05-13 21:38:46,NaN,"<a href=""http://twitter.com/download/iphone"" r...",stephegan613,1,True,False,dyk that we re celebrating years of partnershi...,"(d, y, k, , t, h, a, t, , w, e, , r, e, , ...","(0.5, 0.5)",0.500000,0.5


In [60]:
df_cs.isDuplicated.value_counts()
#Remove unnecessary columns
df_cs.drop(['Unnamed: 0','longitude','favorited','truncated','latitude','id','isDuplicated','replyToUID'],axis=1,inplace=True) 


In [61]:
df_cs['cleaned_text'] = df_cs['text'].apply(lambda x : text_cleaners(x))

C:\Users\Amara\Anaconda2\envs\py36\lib\site-packages\bs4\__init__.py:282: UserWarning: "https://t.co/tkLqMMu4ht" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Users\Amara\Anaconda2\envs\py36\lib\site-packages\bs4\__init__.py:282: UserWarning: "https://t.co/wCn60XGHFy" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Users\Amara\Anaconda2\envs\py36\lib\site-packages\bs4\__init__.py:282: UserWarning: "https://t.co/mbIrenFhri" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beauti

In [62]:
df_cs['blobw'] = df_cs['cleaned_text'].apply(TextBlob)

In [63]:
df_cs['sentiment'] = df_cs['cleaned_text'].apply(sentiment_calc)


df_cs['Polarity'] = df_cs['cleaned_text'].apply(polarity_calc)
df_cs['Subjectivity'] = df_cs['cleaned_text'].apply(subjectivity_calc)

In [73]:
df_cs['Sentiment_class'] = df_cs.apply(conditions, axis=1)

#Export Canadian Tire file to local machine
df_cs.to_csv("costco_senti_score.csv", encoding='utf-8', index=False)

In [66]:
# Get Sobeys for walmart tweet

#Read Sobeys tweets csv file
df_sb = pd.read_csv("sobeys_all.csv", encoding="latin-1")

In [69]:
#check the loaded file
df_sb.head(2)

,Unnamed: 0,text,favorited,favoriteCount,replyToSN,created,truncated,replyToSID,id,replyToUID,statusSource,screenName,retweetCount,isRetweet,retweeted,longitude,latitude,isDuplicated
0,1,@karla_macf_pc @NovaScotiaLive @sobeys Right a...,False,0,karla_macf_pc,2018-05-08 03:18:03,False,9.936283e+17,9.936915e+17,1.154630e+09,"<a href=""http://twitter.com/download/iphone"" r...",valentino_nora,0,False,False,NaN,NaN,False
1,2,@tradegovukCAN @sobeys @SafewayCanada @DanaDic...,False,0,tradegovukCAN,2018-05-08 02:18:47,True,9.928476e+17,9.936766e+17,1.066106e+08,"<a href=""http://twitter.com/download/iphone"" r...",ScotCanBC,0,False,False,NaN,NaN,False


In [70]:
df_sb.isDuplicated.value_counts()
#Remove unnecessary columns
df_sb.drop(['Unnamed: 0','longitude','favorited','truncated','latitude','id','isDuplicated','replyToUID'],axis=1,inplace=True) 


In [71]:
df_sb['cleaned_text'] = df_sb['text'].apply(lambda x : text_cleaners(x))

C:\Users\Amara\Anaconda2\envs\py36\lib\site-packages\bs4\__init__.py:282: UserWarning: "https://t.co/dMJ54sSrRB" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Users\Amara\Anaconda2\envs\py36\lib\site-packages\bs4\__init__.py:282: UserWarning: "https://t.co/yMa8uJ0GUh" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Users\Amara\Anaconda2\envs\py36\lib\site-packages\bs4\__init__.py:282: UserWarning: "https://t.co/WRzdcPaspg" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beauti

In [72]:

df_sb['blobw'] = df_sb['cleaned_text'].apply(TextBlob)

df_sb['sentiment'] = df_sb['cleaned_text'].apply(sentiment_calc)


df_sb['Polarity'] = df_sb['cleaned_text'].apply(polarity_calc)
df_sb['Subjectivity'] = df_sb['cleaned_text'].apply(subjectivity_calc)

In [74]:

df_sb['Sentiment_class'] = df_sb.apply(conditions, axis=1)

#Export Canadian Tire file to local machine
df_sb.to_csv("sobeys_senti_score.csv", encoding='utf-8', index=False)
